### Step 2 - Database Engineering
Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

In [15]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [16]:
#Use Pandas to read your cleaned measurements and stations CSV data.
file1="Resources/clean_hawaii_measurements.csv"
file2="Resources/clean_hawaii_stations.csv"
measurements_df=pd.read_csv(file1)
stations_df=pd.read_csv(file2)

In [17]:
#start index from 1 instead of 0
measurements_df.index+=1
#reset index
measurements_df.reset_index(drop=False, inplace=True)
measurements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19550 entries, 0 to 19549
Data columns (total 5 columns):
index      19550 non-null int64
station    19550 non-null object
date       19550 non-null object
prcp       19550 non-null float64
tobs       19550 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 763.8+ KB


In [18]:
#rename 'index' column to 'id'
measurements_df.rename(columns={'index':'id'}, inplace=True)
measurements_df.head(10)

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65
1,2,USC00519397,2010-01-02,0.00,63
2,3,USC00519397,2010-01-03,0.00,74
3,4,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-06,0.00,73
5,6,USC00519397,2010-01-07,0.06,70
6,7,USC00519397,2010-01-08,0.00,64
7,8,USC00519397,2010-01-09,0.00,68
8,9,USC00519397,2010-01-10,0.00,73
9,10,USC00519397,2010-01-11,0.01,64


In [19]:
#start index from 1 instead of 0
stations_df.index += 1
#reset index
stations_df.reset_index(drop=False, inplace=True)
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
index        9 non-null int64
station      9 non-null object
name         9 non-null object
latitude     9 non-null float64
longitude    9 non-null float64
elevation    9 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 512.0+ bytes


In [20]:
#rename 'index' column to 'id'
stations_df.rename(columns={'index':'id'}, inplace=True)
stations_df.head(10)

,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,7,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [21]:
#Use declarative_base and create ORM classes for each table.
#You will need a class for Measurement and for Station.
#Make sure to define your primary keys.
class Measurement(Base):
    __tablename__='measurement'
    id=Column(Integer, primary_key=True)
    station=Column(String)
    date=Column(String)
    prcp=Column(Float)
    tobs=Column(Integer)

In [22]:
class Station(Base):
    __tablename__='station'
    id=Column(Integer, primary_key=True)
    station=Column(String)
    name=Column(String)
    latitude=Column(Float)
    longitude=Column(Float)
    elevation=Column(Float)

In [23]:
#Use the engine and connection string to create a database called hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [24]:
#Once you have your ORM classes defined, create the tables in the database using create_all
Base.metadata.create_all(engine)

In [25]:
#check tables names
engine.table_names()

['measurement', 'station']

In [26]:
#connect to the database
conn=engine.connect()

In [27]:
#call the to_sql method to convert df to a table in a database.
#set the 'keep_exists'='append' to  insert data into existing tabels.

measurements_df.to_sql("measurement", conn, if_exists="append", index=False)
stations_df.to_sql("station", conn, if_exists="append", index=False)

In [28]:
#We can then verify that everything works by querying the database
engine.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]

In [29]:
engine.execute("select  * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [30]:
conn.close()